In [ ]:
# train in sagemaker
# create .lst files
# create s3 bucket to hold images and lst files

In [ ]:
!pip install kaggle

In [ ]:
import json
import zipfile

!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json
with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia --force

In [ ]:
with zipfile.ZipFile('./chest-xray-pneumonia.zip', 'r') as zip_ref:
    zip_ref.extractall('./data')

In [ ]:
import glob
import random
import matplotlib.pyplot as plt

def get_random_image(dir, condition):
    placeholder = ''
    
    if condition == 'p':
        placeholder = 'PNEUMONIA'
    elif condition == 'n':
        placeholder = 'NORMAL'
    else:
        raise Exception('Sorry invalid condition')
    
    folder = f'./data/chest_xray/{dir}/{placeholder}/*.jpeg'
    img_paths = glob.glob(folder)
    max_len = len(img_paths)
    random_number = random.randint(0, max_len)
    
    for index, item in enumerate(img_paths, start=1):
        if index == random_number:
            print(index, item)
            image = plt.imread(item)
            ready_image = plt.imshow(image)
            plt.show()
            return ready_image

In [ ]:
get_random_image('train', 'p')

In [ ]:
get_random_image('train', 'n')

In [ ]:
from PIL import Image

image = Image.open('data/chest_xray/train/NORMAL/NORMAL2-IM-1423-0001.jpeg')
print(image.format)
print(image.size)
print(image.mode)

In [ ]:
folder = './data/chest_xray/train/*/*.jpeg'

counterPneu = 0
counterNormal = 0

img_paths = glob.glob(folder)

for i in img_paths:
    if 'person' in i:
        full_size_image = Image.open(i)
        img = full_size_image.resize((224, 224))
        plt.imsave(fname='./data/chest_xray/train' + '/train_pneumonia' + str(counterPneu) + '.jpeg', arr=img, format='jpeg', cmap='Greys')
        counterPneu += 1
    else:
        full_size_image = Image.open(i)
        img = full_size_image.resize((224, 224))
        plt.imsave(fname='./data/chest_xray/train' + '/train_normal' + str(counterNormal) + '.jpeg', arr=img, format='jpeg', cmap='Greys')
        counterNormal += 1
        

In [ ]:
folder = './data/chest_xray/test/*/*.jpeg'

counterPneu = 0
counterNormal = 0

img_paths = glob.glob(folder)

for i in img_paths:
    if 'person' in i:
        full_size_image = Image.open(i)
        img = full_size_image.resize((224, 224))
        plt.imsave(fname='./data/chest_xray/test' + '/test_pneumonia' + str(counterPneu) + '.jpeg', arr=img, format='jpeg', cmap='Greys')
        counterPneu += 1
    else:
        full_size_image = Image.open(i)
        img = full_size_image.resize((224, 224))
        plt.imsave(fname='./data/chest_xray/test' + '/test_normal' + str(counterNormal) + '.jpeg', arr=img, format='jpeg', cmap='Greys')
        counterNormal += 1

In [ ]:
folder = './data/chest_xray/val/*/*.jpeg'

counterPneu = 0
counterNormal = 0

img_paths = glob.glob(folder)

for i in img_paths:
    if 'person' in i:
        full_size_image = Image.open(i)
        img = full_size_image.resize((224, 224))
        plt.imsave(fname='./data/chest_xray/val' + '/val_pneumonia' + str(counterPneu) + '.jpeg', arr=img, format='jpeg', cmap='Greys')
        counterPneu += 1
    else:
        full_size_image = Image.open(i)
        img = full_size_image.resize((224, 224))
        plt.imsave(fname='./data/chest_xray/val' + '/val_normal' + str(counterNormal) + '.jpeg', arr=img, format='jpeg', cmap='Greys')
        counterNormal += 1

In [ ]:
import pandas as pd

folder = './data/chest_xray/*/*.jpeg'

category = []
condition_of_lung = []
filenames = []

all_files = glob.glob(folder)

for filename in all_files:
    if 'train' in filename:
        if 'pneumonia' in filename:
            category.append('train')
            filenames.append(filename)
            condition_of_lung.append('pneumonia')
        elif 'normal' in filename:
            category.append('train')
            filenames.append(filename)
            condition_of_lung.append('normal')
    
    if 'test' in filename:
        if 'pneumonia' in filename:
            category.append('test')
            filenames.append(filename)
            condition_of_lung.append('pneumonia')
        elif 'normal' in filename:
            category.append('test')
            filenames.append(filename)
            condition_of_lung.append('normal')
            
    if 'val' in filename:
        if 'pneumonia' in filename:
            category.append('val')
            filenames.append(filename)
            condition_of_lung.append('pneumonia')
        elif 'normal' in filename:
            category.append('val')
            filenames.append(filename)
            condition_of_lung.append('normal')
            
all_data_df = pd.DataFrame({"dataset type": category, "x_ray result": condition_of_lung, "filename": filenames})
print(all_data_df.head())

In [ ]:
import seaborn as sns

g = sns.catplot(x='x_ray result', col='dataset type', kind='count', palette='ch:.55', data=all_data_df, legend=True)
for i in range(0, 3):
    ax = g.facet_axis(0, i)
    for p in ax.patches:
        ax.text(p.get_x() + 0.3, p.get_height() + 1.05, '{0:.0f}'.format(p.get_height()), color='black', rotation='horizontal', size='large')
plt.show()

### Create LST files

In [ ]:
import os

train_folder = './data/chest_xray/train/*.jpeg'
train_df = pd.DataFrame(columns=['labels', 's3_path'], dtype=object)
train_imgs_path = glob.glob(train_folder)
counter = 0
class_arg = ''

for i in train_imgs_path:
    if 'pneumonia' in i:
        class_arg = 1
    else:
        class_arg = 0
    train_df.loc[counter] = [class_arg, os.path.basename(i)]
    counter += 1

print(train_df.head())
plt.show()

In [ ]:
import os

test_folder = './data/chest_xray/test/*.jpeg'
test_df = pd.DataFrame(columns=['labels', 's3_path'], dtype=object)
test_imgs_path = glob.glob(test_folder)
counter = 0
class_arg = ''

for i in test_imgs_path:
    if 'pneumonia' in i:
        class_arg = 1
    else:
        class_arg = 0
    test_df.loc[counter] = [class_arg, os.path.basename(i)]
    counter += 1

print(test_df.head())
plt.show()

In [ ]:
import os

val_folder = './data/chest_xray/val/*.jpeg'
val_df = pd.DataFrame(columns=['labels', 's3_path'], dtype=object)
val_imgs_path = glob.glob(val_folder)
counter = 0
class_arg = ''

for i in val_imgs_path:
    if 'pneumonia' in i:
        class_arg = 1
    else:
        class_arg = 0
    val_df.loc[counter] = [class_arg, os.path.basename(i)]
    counter += 1

print(val_df.head())
plt.show()

In [ ]:
def save_to_lst(df, prefix):
    return df[['labels', 's3_path']].to_csv(f"{prefix}.lst", sep='\t', index=True, header=False)

save_to_lst(train_df.copy(), 'train')
save_to_lst(test_df.copy(), 'test')
# save_to_lst(val_df.copy(), 'val')

### Upload data to S3 bucket

In [4]:
import os

In [5]:
os.environ["DEFAULT_S3_BUCKET"] = bucket

In [ ]:
!aws s3 sync ./data/chest_xray/train s3://${DEFAULT_S3_BUCKET}/train/

In [ ]:
!aws s3 sync ./data/chest_xray/test s3://${DEFAULT_S3_BUCKET}/test/

In [ ]:
import boto3

boto3.Session().resource('s3').Bucket(bucket).Object('train.lst').upload_file('./train.lst')
boto3.Session().resource('s3').Bucket(bucket).Object('test.lst').upload_file('./test.lst')

### Train model

In [2]:
import boto3

In [6]:
import sagemaker
from sagemaker import image_uris
import boto3
from sagemaker import get_execution_role

sess = sagemaker.Session()
algorithm_image = image_uris.retrieve(
    region=boto3.Session().region_name,
    framework='image-classification'
)
print(algorithm_image)

811284229777.dkr.ecr.us-east-1.amazonaws.com/image-classification:1


In [7]:
role = get_execution_role()
print(role)

arn:aws:iam::150407749717:role/service-role/AmazonSageMaker-ExecutionRole-20221004T215064


In [8]:
img_classifier_model = sagemaker.estimator.Estimator(
    algorithm_image,
    role=role,
    instance_count=1,
    instance_type='ml.p2.xlarge',
    volume_size=50,
    input_mode='File',
    max_run=432000,
    output_path=s3_output_location,
    sagemaker_session=sess
)

print(img_classifier_model)

In [10]:
import glob
count = 0
for filepath in glob.glob('./data/chest_xray/train/*.jpeg'):
    count += 1

print(count)

0


In [11]:
img_classifier_model.set_hyperparameters(
    image_shape='3,224,224',
    num_classes=2,
    augmentation_type='crop',
    use_pretrained_model=1,
    num_training_samples=count,
    epochs=5,
    early_stopping=True,
    early_stopping_min_epochs=8,
    early_stopping_patience=5,
    early_stopping_tolerance=0.0,
    lr_scheduler_factor=0.1,
    lr_scheduler_step='8,10,12'
)

print(img_classifier_model.hyperparameters())

{'image_shape': '3,224,224', 'num_classes': 2, 'augmentation_type': 'crop', 'use_pretrained_model': 1, 'num_training_samples': 0, 'epochs': 5, 'early_stopping': True, 'early_stopping_min_epochs': 8, 'early_stopping_patience': 5, 'early_stopping_tolerance': 0.0, 'lr_scheduler_factor': 0.1, 'lr_scheduler_step': '8,10,12'}


In [12]:
from sagemaker.tuner import CategoricalParameter, ContinuousParameter, HyperparameterTuner

hyperparameter_ranges = {
    "learning_rate":ContinuousParameter(0.01, 0.1),
    "mini_batch_size": CategoricalParameter([8, 16, 32]),
    "optimizer":CategoricalParameter(["sgd", 'adam']),
}

In [13]:
objective_metric_name = 'validation:accuracy'
objective_type = 'Maximize'
max_jobs = 5
max_parallel_jobs = 1

In [14]:
tuner = HyperparameterTuner(estimator=img_classifier_model,
                           objective_metric_name=objective_metric_name,
                           hyperparameter_ranges=hyperparameter_ranges,
                           objective_type=objective_type,
                           max_jobs=max_jobs,
                           max_parallel_jobs=max_parallel_jobs,
                           )
print(tuner)

In [15]:
from sagemaker.session import TrainingInput

model_inputs = {
    "train":sagemaker.inputs.TrainingInput(s3_data=f's3://{bucket}/train/', content_type='application/x-image'),
    "validation":sagemaker.inputs.TrainingInput(s3_data=f's3://{bucket}/test/', content_type='application/x-image'),
    "train_lst":sagemaker.inputs.TrainingInput(s3_data=f's3://{bucket}/train.lst', content_type='application/x-image'),
    "validation_lst":sagemaker.inputs.TrainingInput(s3_data=f's3://{bucket}/test.lst', content_type='application/x-image'),
}

print(model_inputs)

{'train': <sagemaker.inputs.TrainingInput object at 0x7fb1c0150350>, 'validation': <sagemaker.inputs.TrainingInput object at 0x7fb1c0150410>, 'train_lst': <sagemaker.inputs.TrainingInput object at 0x7fb1c01503d0>, 'validation_lst': <sagemaker.inputs.TrainingInput object at 0x7fb1c0150450>}


In [ ]:
import time

job_name_prefix = 'classifier'
timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
job_name = job_name_prefix + timestamp

In [ ]:
tuner.fit(inputs=model_inputs,
         job_name=job_name,
         logs=True)

In [16]:
import sagemaker
from sagemaker import get_execution_role

In [17]:
role = get_execution_role()

In [19]:
model = sagemaker.model.Model(
image_uri=algorithm_image,
model_data="classifier-2022-10-14-01-50-55-001-a97130b2/output/model.tar.gz",
role=role)

In [20]:
deployment=model.deploy(
initial_instance_count=1,
instance_type='ml.m4.xlarge',
endpoint_name=endpoint_name
)

---------!

In [21]:
from sagemaker.predictor import Predictor
predictor = Predictor(endpoint_name)

In [22]:
from sagemaker.serializers import IdentitySerializer
import base64

file_name = './data/chest_xray/val/val_pneumonia0.jpeg'

predictor.serializer = IdentitySerializer('image/jpeg')
with open(file_name, 'rb') as f:
    payload = f.read()

inference = predictor.predict(data=payload)
print(inference)

b'[0.0014596829423680902, 0.9985402822494507]'
